In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from romav2 import RoMaV2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model once
roma = RoMaV2().to(device).eval()

Loading RoMa v2 weights from local path: ../../../../../../../../scratch/dynrecon/checkpoints/romav2.pt


Using cache found in /home/dynrecon/.cache/torch/hub/facebookresearch_dinov3_adc254450203739c8149213a7a69d8d905b4fcfa


2026-01-22 11:09:13 INFO     romav2.romav2 - romav2:116 in __init__ - Compiling RoMa v2...

2026-01-22 11:09:15 INFO     romav2.romav2 - romav2:118 in __init__ - RoMa v2 initialized.

In [ ]:
import os

QUERY_PATH = "../../../../../../../scratch/toponavgroup/indoor-topo-loc/datasets/rrc-lab-data/wheelchair-runs-20241220/run-2-wheelchair-query"
REF_PATH = "../../../../../../../scratch/toponavgroup/indoor-topo-loc/datasets/rrc-lab-data/wheelchair-runs-20241220/run-1-wheelchair-mapping"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def visualize_dense_matches(q_idx, ref_idx, num_samples=10000):
    # 1. Load Images

    q_filename = f"{q_idx}.png" 
    r_filename = f"{ref_idx}.png"

    q_path = os.path.join(QUERY_PATH, "rgb", q_filename)
    r_path = os.path.join(REF_PATH, "rgb", r_filename)

    # --- DEBUG: Check if files actually exist ---
    if not os.path.exists(q_path):
        print(f"ERROR: Query image not found at: {os.path.abspath(q_path)}")
        return
    if not os.path.exists(r_path):
        print(f"ERROR: Ref image not found at: {os.path.abspath(r_path)}")
        return

    # q_path = f"{QUERY_PATH}/rgb/{q_idx}.png"
    # r_path = f"{REF_PATH}/rgb/{ref_idx}.png"

    img1 = cv2.cvtColor(cv2.imread(q_path), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(r_path), cv2.COLOR_BGR2RGB)
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # 2. Inference
    with torch.inference_mode():
        preds = roma.match(q_path, r_path)
        # Sample a large number of matches for a "dense" look
        matches, _, _, _ = roma.sample(preds, num_samples)
        
        kpts_q, kpts_ref = roma.to_pixel_coordinates(matches, h1, w1, h2, w2)
        kpts_q = kpts_q.cpu().numpy()
        kpts_ref = kpts_ref.cpu().numpy()

    # 3. Generate Colors based on Query Positions (HSV mapping)
    # Normalized x and y to [0, 1]
    norm_x = kpts_q[:, 0] / w1
    norm_y = kpts_q[:, 1] / h1

    # Create colors: Hue based on X, Saturation based on Y
    colors = np.zeros((len(kpts_q), 3))
    colors[:, 0] = norm_x * 180  # Hue (0-180 in OpenCV)
    colors[:, 1] = norm_y * 255  # Saturation
    colors[:, 2] = 255           # Value
    
    # Convert HSV to RGB for plotting
    colors_uint8 = colors.reshape(-1, 1, 3).astype(np.uint8)
    colors_rgb = cv2.cvtColor(colors_uint8, cv2.COLOR_HSV2RGB).reshape(-1, 3) / 255.0

    # 4. Plotting
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
    
    # Query Image
    ax[0].imshow(img1)
    ax[0].scatter(kpts_q[:, 0], kpts_q[:, 1], c=colors_rgb, s=2, alpha=0.5)
    ax[0].set_title(f"Query Image ({q_idx}) - Colors assigned by Position")
    ax[0].axis('off')

    # Reference Image
    ax[1].imshow(img2)
    ax[1].scatter(kpts_ref[:, 0], kpts_ref[:, 1], c=colors_rgb, s=2, alpha=0.5)
    ax[1].set_title(f"Ref Image ({ref_idx}) - Matching Color points")
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

# Example Usage:
# visualize_dense_matches(100, 105, num_samples=15000)

In [10]:
visualize_dense_matches(1140, 1713, num_samples=15000)

ERROR: Query image not found at: /home/scratch/toponavgroup/indoor-topo-loc/datasets/rrc-lab-data/wheelchair-runs-20241220/run-2-wheelchair-query/rgb/1140.png
